In [ ]:
# 1️⃣ Import required libraries and define project paths

import pandas as pd
import numpy as np
from pathlib import Path


In [ ]:
# 2️⃣ Load the Home Credit training dataset

PROJECT_DIR = Path("~/Documents/credit-scoring-home-credit").expanduser()
DATA_RAW = PROJECT_DIR / "data" / "raw"

train_path = DATA_RAW / "application_train.csv"

df = pd.read_csv(train_path, low_memory=False)

df.shape


In [ ]:
# 3️⃣ Preview the first few rows of the dataset

df.head(3)


In [ ]:
# 4️⃣ Examine the target distribution (class imbalance check)

target_counts = df["TARGET"].value_counts()
target_rate = df["TARGET"].value_counts(normalize=True)

target_counts, target_rate


In [ ]:
# 5️⃣ Demonstrate why accuracy is misleading in imbalanced datasets

# Simulate a naive model that predicts everyone as non-default
y_true = df["TARGET"].values
y_pred_all_zero = np.zeros_like(y_true)

accuracy_all_zero = (y_pred_all_zero == y_true).mean()
default_recall_all_zero = (
    ((y_pred_all_zero == 1) & (y_true == 1)).sum()
    / max((y_true == 1).sum(), 1)
)

accuracy_all_zero, default_recall_all_zero


In [ ]:
# 6️⃣ Inspect data types (numeric vs categorical overview)

df.dtypes.value_counts()


In [ ]:
# 7️⃣ Calculate percentage of missing values per column (top 25)

missing_pct = df.isna().mean().sort_values(ascending=False) * 100
missing_pct.head(25).round(2)


In [ ]:
# 8️⃣ Count how many columns exceed different missingness thresholds

thresholds = [10, 20, 40, 60, 80]
missing_summary = {t: (missing_pct > t).sum() for t in thresholds}

missing_summary


In [ ]:
# 9️⃣ Generate summary statistics for numeric variables

df.describe().T[["mean", "std", "min", "max"]].head(15)


In [ ]:
# 🔟 Separate numeric and categorical columns for later analysis

num_cols = df.select_dtypes(include=["number"]).columns.tolist()
cat_cols = df.select_dtypes(exclude=["number"]).columns.tolist()

len(num_cols), len(cat_cols)


In [ ]:
# 1️⃣1️⃣ Analyze default rate by a categorical feature (example: CODE_GENDER)

col = "CODE_GENDER"

default_by_gender = (
    df.groupby(col)["TARGET"]
    .agg(default_rate="mean", count="size")
    .sort_values("default_rate", ascending=False)
)

default_by_gender


In [ ]:
# 1️⃣2️⃣ Compare a numeric variable across default vs non-default borrowers (example: AMT_INCOME_TOTAL)

col = "AMT_INCOME_TOTAL"

df.groupby("TARGET")[col].describe()[["mean", "50%", "std", "min", "max"]]


In [ ]:
# 1️⃣3️⃣ Compute correlation of numeric features with TARGET

num_df = df[num_cols].copy()

corr_with_target = (
    num_df.corr(numeric_only=True)["TARGET"]
    .drop("TARGET")
    .sort_values()
)

corr_with_target.head(10), corr_with_target.tail(10)


In [ ]:
# 1️⃣4️⃣ Identify columns with more than 60% missing

missing_pct = df.isna().mean()

high_missing_cols = missing_pct[missing_pct > 0.6].index.tolist()

len(high_missing_cols), high_missing_cols[:10]


In [ ]:
# 1️⃣5️⃣ Check if missingness predicts default

missing_analysis = []

for col in high_missing_cols:
    
    temp = pd.DataFrame({
        "is_missing": df[col].isna().astype(int),
        "TARGET": df["TARGET"]
    })
    
    grouped = temp.groupby("is_missing")["TARGET"].mean()
    
    if len(grouped) == 2:  # both missing and non-missing exist
        missing_analysis.append({
            "column": col,
            "default_rate_not_missing": grouped[0],
            "default_rate_missing": grouped[1],
            "difference": grouped[1] - grouped[0]
        })

missing_df = pd.DataFrame(missing_analysis).sort_values("difference", ascending=False)

missing_df.head(10)


In [ ]:
# 1️⃣6️⃣ Identify high-missing columns (>60%)

missing_pct = df.isna().mean()

high_missing_cols = missing_pct[missing_pct > 0.6].index.tolist()

len(high_missing_cols)


In [ ]:
# 1️⃣7️⃣ Create missing indicator features

df_enhanced = df.copy()

for col in high_missing_cols:
    indicator_name = col + "_MISSING"
    df_enhanced[indicator_name] = df[col].isna().astype(int)

df_enhanced[[high_missing_cols[0], high_missing_cols[0] + "_MISSING"]].head()


In [ ]:
# 1️⃣8️⃣ Drop raw columns with >60% missing

df_enhanced = df_enhanced.drop(columns=high_missing_cols)

df_enhanced.shape


In [ ]:
# 1️⃣9️⃣ Recompute missingness after cleaning

missing_after = df_enhanced.isna().mean().sort_values(ascending=False) * 100
missing_after.head(20).round(2)


In [ ]:
# 2️⃣0️⃣ Remove redundant _MEDI and _MODE versions (keep _AVG only)

cols_to_drop_redundant = [
    col for col in df_enhanced.columns
    if col.endswith("_MEDI") or col.endswith("_MODE")
]

len(cols_to_drop_redundant)
df_enhanced = df_enhanced.drop(columns=cols_to_drop_redundant)

df_enhanced.shape


In [ ]:
(df_enhanced.isna().mean().sort_values(ascending=False) * 100).head(15)


In [ ]:
# 2️⃣1️⃣ Drop remaining high-missing property features (>45%) except EXT_SOURCE and OCCUPATION_TYPE

cols_to_keep_even_if_missing = [
    "EXT_SOURCE_1",
    "EXT_SOURCE_3",
    "OCCUPATION_TYPE"
]

remaining_missing = df_enhanced.isna().mean()

cols_to_drop_high_missing = [
    col for col in remaining_missing[remaining_missing > 0.45].index
    if col not in cols_to_keep_even_if_missing
]

len(cols_to_drop_high_missing)
df_enhanced = df_enhanced.drop(columns=cols_to_drop_high_missing)

df_enhanced.shape


In [ ]:
(df_enhanced.isna().mean().sort_values(ascending=False) * 100).head(15)


In [ ]:
# 2️⃣1️⃣.5️⃣ Fix DAYS_EMPLOYED anomaly (365243 means missing)

df_enhanced["DAYS_EMPLOYED"] = df_enhanced["DAYS_EMPLOYED"].replace(365243, np.nan)


In [ ]:
# 2️⃣2️⃣ Prepare features and target

X = df_enhanced.drop(columns=["TARGET"])
y = df_enhanced["TARGET"].astype(int)


In [ ]:
# 2️⃣3️⃣ Train / validation split (stratified)

from sklearn.model_selection import train_test_split

X_train, X_valid, y_train, y_valid = train_test_split(
    X, y,
    test_size=0.2,
    random_state=42,
    stratify=y
)

X_train.shape, X_valid.shape


In [ ]:
# 2️⃣4️⃣ Separate numeric and categorical columns

num_cols = X_train.select_dtypes(include=["number"]).columns.tolist()
cat_cols = X_train.select_dtypes(exclude=["number"]).columns.tolist()

len(num_cols), len(cat_cols)


In [ ]:
# 2️⃣5️⃣ Build preprocessing pipeline (impute + scale numeric, impute + one-hot categorical)

from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.impute import SimpleImputer

# Numeric: median impute + standardize (helps logistic regression converge)
numeric_pipeline = Pipeline(steps=[
    ("imputer", SimpleImputer(strategy="median")),
    ("scaler", StandardScaler())
])

# Categorical: most-frequent impute + one-hot encode
categorical_pipeline = Pipeline(steps=[
    ("imputer", SimpleImputer(strategy="most_frequent")),
    ("encoder", OneHotEncoder(handle_unknown="ignore"))
])

# Combine numeric + categorical preprocessing
preprocessor = ColumnTransformer(
    transformers=[
        ("num", numeric_pipeline, num_cols),
        ("cat", categorical_pipeline, cat_cols)
    ],
    remainder="drop"
)


In [ ]:
# 2️⃣6️⃣ Define logistic regression model (baseline PD model)

from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline

logit_model = LogisticRegression(
    max_iter=3000,          # increased to help convergence
    class_weight="balanced",# handle class imbalance
    solver="lbfgs"
)

# Full pipeline = preprocessing + model
clf = Pipeline(steps=[
    ("preprocessor", preprocessor),
    ("model", logit_model)
])


In [ ]:
# 2️⃣7️⃣ Train the model on the training set

clf.fit(X_train, y_train)


In [ ]:
# 2️⃣8️⃣ Predict PD (probability of default) on validation set and compute ROC-AUC / PR-AUC

from sklearn.metrics import roc_auc_score, average_precision_score

# Predicted probability of default (PD)
p_valid = clf.predict_proba(X_valid)[:, 1]

roc_auc = roc_auc_score(y_valid, p_valid)
pr_auc = average_precision_score(y_valid, p_valid)

roc_auc, pr_auc


In [ ]:
# 2️⃣9️⃣ Confusion matrix at a default threshold of 0.50 (for illustration only)

from sklearn.metrics import confusion_matrix

pred_05 = (p_valid >= 0.5).astype(int)
confusion_matrix(y_valid, pred_05)


In [ ]:
# 3️⃣0️⃣ Classification report at threshold 0.50 (precision/recall for default class)

from sklearn.metrics import classification_report

print(classification_report(y_valid, pred_05, digits=4))


In [ ]:
# 3️⃣1️⃣ Plot ROC curve and PR curve (visual model performance)

import matplotlib.pyplot as plt
from sklearn.metrics import RocCurveDisplay, PrecisionRecallDisplay

plt.figure()
RocCurveDisplay.from_predictions(y_valid, p_valid)
plt.title("ROC Curve (Validation)")
plt.show()

plt.figure()
PrecisionRecallDisplay.from_predictions(y_valid, p_valid)
plt.title("Precision-Recall Curve (Validation)")
plt.show()


In [ ]:
# 3️⃣2️⃣ Calibration curve (are predicted PDs aligned with observed default rates?)
from pathlib import Path

PROJECT_DIR = Path("~/Documents/credit-scoring-home-credit").expanduser()
PLOTS_DIR = PROJECT_DIR / "plots"
PLOTS_DIR.mkdir(parents=True, exist_ok=True)

PLOTS_DIR
from sklearn.calibration import calibration_curve

frac_pos, mean_pred = calibration_curve(y_valid, p_valid, n_bins=10)


plt.plot(mean_pred, frac_pos, marker="o")
plt.plot([0, 1], [0, 1], linestyle="--")
plt.xlabel("Mean Predicted PD")
plt.ylabel("Observed Default Rate")
plt.title("Calibration Curve (Validation)")
plt.savefig(PLOTS_DIR / "calibration_curve_validation.png", dpi=300, bbox_inches="tight")


In [ ]:
# 3️⃣3️⃣ Compute KS statistic (common in credit risk)

# KS = max difference between CDFs of scores for good vs bad
import numpy as np

def ks_statistic(y_true, y_score, n_bins=100):
    data = pd.DataFrame({"y": y_true, "score": y_score}).sort_values("score")
    data["bin"] = pd.qcut(data["score"], q=n_bins, duplicates="drop")
    
    grouped = data.groupby("bin")["y"]
    bad_rate = grouped.mean()
    total = grouped.size()
    
    bad_cum = (bad_rate * total).cumsum() / (data["y"].sum())
    good_cum = ((1 - bad_rate) * total).cumsum() / ((1 - data["y"]).sum())
    
    ks = np.max(np.abs(bad_cum - good_cum))
    return ks

ks = ks_statistic(y_valid, p_valid)
ks


In [ ]:
# 3️⃣4️⃣ Extract logistic regression coefficients and compute odds ratios

# Get feature names after preprocessing
feature_names = clf.named_steps["preprocessor"].get_feature_names_out()

# Get coefficients
coefficients = clf.named_steps["model"].coef_[0]

coef_df = pd.DataFrame({
    "feature": feature_names,
    "coefficient": coefficients,
    "odds_ratio": np.exp(coefficients)
})

coef_df_sorted = coef_df.sort_values("coefficient")

coef_df_sorted.head(10), coef_df_sorted.tail(10)
